Assuming we are using data from here for investors and companies

investors: https://www.openvc.app/investor-lists/pre-seed-investors

companies: https://startups.gallery/

For Companies.

In [3]:
places = ["United States", "United Kingdom", "Sweden", "Spain", "South Korea", "Singapore", "Poland", "Unknown"]
stages = ["Pre-seed", "Seed", "Series A", "Series B", "Series C", "Series D", "Unknown"]
industries = ["Aerospace", "AI", "Analytics", "Biotech", "Climate", "Construction", "Unknown"]
work_types = ["Remote", "Hybrid", "Onsite", "Unknown"]


def safe_index(value, lst):
    return lst.index(value) if value in lst else len(lst) - 1

def company_profile(name, desc, place, stage, industry, work_type, funding):

  profile = {"name": name,
            "desc": desc,
            "place": safe_index(place, places),
            "stage": safe_index(stage, stages),
            "industries": [safe_index(i, industries) for i in industry],
            "work_type": safe_index(work_type, work_types),
            "funding": funding
            }

  return profile

For investors.

In [4]:
places = ["United States", "United Kingdom", "Sweden", "Spain", "South Korea", "Singapore", "Poland", "Unknown"]
stages = ["Pre-seed", "Seed", "Series A", "Series B", "Series C", "Series D", "Unknown"]
industries = ["Aerospace", "AI", "Analytics", "Biotech", "Climate", "Construction", "Unknown"]
work_types = ["Remote", "Hybrid", "Onsite", "Unknown"]


def safe_index(value, lst):
    return lst.index(value) if value in lst else len(lst) - 1

def investor_profile(name, desc, i_place, i_stage, i_industry, check_size_bot, check_size_top):

  profile = {"name": name,
            "desc": desc,
            "places": [safe_index(i_p, places) for i_p in i_place],
            "stages": [safe_index(i_s, stages) for i_s in i_stage],
            "industries": [safe_index(i_i, industries) for i_i in i_industry],
            "check_size": [check_size_bot, check_size_top]
             }

  return profile

Sample Companies + Sample Investors

In [21]:
company_1 = company_profile(
    name = "Mandolin",
    desc = "Nearly every disease will become treatable in our lifetimes."
           "Mandolin is laying the clinical and financial infrastructure to get groundbreaking treatments to patients faster,"
           "powered by AI agents. Mandolin partners closely with the largest healthcare institutions in the US, covering more than $10B drug spend across the country.",
    place = "United States",
    stage = "Series A",
    industry = "AI",
    work_type = "Onsite",
    funding = 40 * (10 ** 6)
)

company_2 = company_profile(
    name = "Durin",
    desc = "Durin builds and operates automated drill rigs to accelerate mineral discovery."
           "We believe that raw materials should not be a constraint on human progress, and are actively developing the technologies to make this possible,"
           "with the ultimate goal of becoming the bedrock of the global supply chain.",
    place = "United States",
    stage = "Pre-seed",
    industry = "Biotech",
    work_type = "Onsite",
    funding = 3.4 * (10 ** 6)
)

company_3 = company_profile(
    name = "Variant",
    desc = "Variant generates software that’s actually well designed."
           "Instead of a confined conversation, it creates endless designs from a single idea."
           "You can freely explore, discover directions you wouldn’t have thought of, and build better software as a result.",
    place = "Seed",
    stage = "Pre-seed",
    industry = "Climate",
    work_type = "Onsite",
    funding = 3.4 * (10 ** 6)
)

companies = [company_1, company_2, company_3]


### AI Generated Investor Templates

investor_1 = investor_profile(
    name = "NorthBridge Angels",
    desc = "We invest in early-stage teams that have developed a scalable business idea with initial traction and a defensible technology.",
    i_place = ["United States", "United Kingdom"],
    i_stage = ["Seed", "Series A"],
    i_industry = ["AI", "Analytics"],
    check_size_bot = 100_000,
    check_size_top = 1_000_000
)

investor_2 = investor_profile(
    name = "Catalyst Climate Fund",
    desc = "We invest in AI & tech enabled StartUp's & industry agnostic."
           "We are a high-conviction, tech-enabled Pre-seed / seed-stage venture fund investing globally, with a strong emphasis on the India-US corridor.",
    i_place = ["Spain", "Sweden", "United States"],
    i_stage = ["Pre-seed", "Seed"],
    i_industry = ["Climate", "Aerospace", "Construction"],
    check_size_bot = 250_000,
    check_size_top = 2_500_000
)

investor_3 = investor_profile(
    name = "Frontier Bio Partners",
    desc = "We invest in impact startups primarily in the Nordics."
           "Investment Themes: Energy Transition, Circularity & Sustainable Food Systems."
           "Investment Stage: Late Seed to Series A.",
    i_place = ["United Kingdom", "Poland", "Singapore"],
    i_stage = ["Seed", "Series A", "Series B"],
    i_industry = ["Biotech", "Analytics"],
    check_size_bot = 1_000_000,
    check_size_top = 10_000_000
)

investors = [investor_1, investor_2, investor_3]

Now... I'll think about how to use a Light Gradient Booster Under This Context.

At the end of the day, we would want a table that looks something like this:

| investor_id | company_id | overlap_industries | place_ fit | stage_fit | money_gap | text_similarity | label |
|--------------|-------------|--------------------|------------|------------|------------|------------------|--------|
| 1 | 10 | 0.8 | 1 | 1 | +500,000 | 0.72 | 1 |
| 1 | 11 | 0.3 | 0 | 1 | -1,500,000 | 0.41 | 0 |

- overlap_industries: Jaccard or overlap score between investor and company industries.  
- stage_fit: 1 if investor’s stage preference matches the company’s funding stage.  
- place_fit: same idea, 1 if investor's preference matches the company place
- money_gap: Difference between investor’s available check size and company’s typical stage median.  
- text_similarity: Cosine similarity between investor’s bio and company’s description (via BERT embeddings).  
- label: 1 = good match (invested/interested), 0 = investor not interested, 2 = investor has not known company yet

In [7]:
def jaccard(investor_industries, company_industries): # used for overlapping industries
    A, B = set(investor_industries), set(company_industries)
    if not A and not B:
        return 0.0
    return len(A & B) / float(len(A | B))

def stage_fit(investor_stages, company_stage):
    return int(company_stage in investor_stages and company_stage != "Unknown")

def place_fit(investor_places, company_place):
    return int(company_place in investor_places and company_place != "Unknown")


how I'll deal with the text data, through sentence transformers.

https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2


In [8]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

In [18]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def encode_text(sentences, tokenizer, model):
    encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
      model_output = model(**encoded_input)
    embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    embeddings = F.normalize(embeddings, p=2, dim=1)
    return embeddings

def text_similarity(embeddings):
    sim = F.cosine_similarity(embeddings[0], embeddings[1], dim=0) # compare sentence 1 with sentence 2
    return sim.item()

don't really know how to deal with the money... so let's pretend that doesn't exist for now.

In [23]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

def investor_company_similarity(investor, company):
    row = {
        "investor_name": investor["name"],
        "company_name": company["name"],
        "overlap_industries": jaccard(investor["industries"], company["industries"]),
         "place_fit": place_fit(investor["places"], company["place"]),
         "stage_fit": stage_fit(investor["stages"], company["stage"]),
         "text_similarity": text_similarity(encode_text([investor["desc"], company["desc"]], tokenizer, model))}
    return row

In [24]:
row = investor_company_similarity(investor_1, company_1)
row

{'investor_name': 'NorthBridge Angels',
 'company_name': 'Mandolin',
 'overlap_industries': 0.0,
 'place_fit': 1,
 'stage_fit': 1,
 'text_similarity': 0.2565643787384033}

Great! But now we needed to do this for all companies and all investors... which means 1 big table?

Or at least, all companies and all investors that pass a certain few metrics (so then it's worth to do more computations with them)

In [33]:
import pandas as pd
from itertools import product
import random

def build_feature_table(investors, companies):
    rows = []
    for investor, company in product(investors, companies):
        row = investor_company_similarity(investor, company)
        rows.append(row)
    feature_table = pd.DataFrame(rows)
    return feature_table

feature_table = build_feature_table(investors, companies)
random.seed(40)
feature_table["label"] = [random.randint(0, 2) for _ in range(len(featue_table))]
feature_table

,investor_name,company_name,overlap_industries,place_fit,stage_fit,text_similarity,label
0,NorthBridge Angels,Mandolin,0.0,1,1,0.256564,1
1,NorthBridge Angels,Durin,0.0,1,0,0.211503,2
2,NorthBridge Angels,Variant,0.0,0,0,0.181078,2
3,Catalyst Climate Fund,Mandolin,0.0,1,0,0.235770,0
4,Catalyst Climate Fund,Durin,0.0,1,1,0.244420,0
5,Catalyst Climate Fund,Variant,0.0,0,1,0.122694,1
6,Frontier Bio Partners,Mandolin,0.0,0,1,0.180860,2
7,Frontier Bio Partners,Durin,0.0,0,0,0.174860,2
8,Frontier Bio Partners,Variant,0.0,0,0,0.055046,2


There sould be more control over if we need to do a text similairty comparison or not depending if previous criterias seem to have met or not, but that's for another day.

Welcome to knowing nothing about Light Gradient Boosters... apparently it's a context based reccomendation model that soehow works in this scenario.

All below code is GPT cuz I have no idea how light gradient boosters work this is just a rec system made by microsoft

In [42]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

# Keep rows with label 0 or 1 for training
train_data = feature_table[feature_table["label"].isin([0, 1])].copy()

feature_columns = ["overlap_industries", "place_fit", "stage_fit", "text_similarity"]
X = train_data[feature_columns]
y = train_data["label"]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)

train_dataset = lgb.Dataset(X_train, label=y_train)
valid_dataset = lgb.Dataset(X_valid, label=y_valid)

params = {
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.05,
    "num_leaves": 31,
    "verbose": -1
}

model = lgb.train(
    params,
    train_set=train_dataset,
    valid_sets=[valid_dataset],
    valid_names=["valid"],
    num_boost_round=200,
)

score the unseen ones

In [43]:
candidates = feature_table[feature_table["label"] == 2].copy()
candidates["score"] = model.predict(candidates[feature_columns])

top_recommendations = (
    candidates.sort_values(["investor_name", "score"], ascending=[True, False])
              .groupby("investor_name")
              .head(5)[["investor_name", "company_name", "score"]]
              .reset_index(drop=True)
)

top_recommendations

,investor_name,company_name,score
0,Frontier Bio Partners,Mandolin,0.5
1,Frontier Bio Partners,Durin,0.5
2,Frontier Bio Partners,Variant,0.5
3,NorthBridge Angels,Durin,0.5
4,NorthBridge Angels,Variant,0.5
